In [22]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [23]:
## PREPARING DATASET

data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['Price']=housing.target
data.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [24]:
from urllib.parse import urlparse
## Indepenedent and Dependednt features

x = data.drop(columns=["Price"])
y = data["Price"]

In [25]:
## HYPERPARAMETER TUNING USING GRID SEARCH CV

def hyperparameter(x_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator= rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2, scoring = "neg_mean_squared_error")

    grid_search.fit(x_train, y_train)
    return grid_search

In [27]:
## Split the data into training and test sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

from mlflow.models import infer_signature

signature = infer_signature(x_train, y_train)


## DEFINE THE HYPERPARAMETER GRID

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

## Start the MLFLOW Experiment

with mlflow.start_run():
    ## PERFORM HYPERPARAMETER TUNING
    grid_search = hyperparameter(x_train, y_train, param_grid)

    ## Get the best model
    best_model=grid_search.best_estimator_

    ## Evaluate the best model
    y_pred = best_model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)

    ## Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_n_estimators", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_n_estimators", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_n_estimators", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)

    ## Tracking URL 

    mlflow.set_tracking_uri(uri="https://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, "model", registered_model_name = "Best RandomForest Model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature = signature)

    print(f"Best hyper parameters: {grid_search.best_params_}")
    print(f"Mean squared error: {mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


MlflowException: Changing param values is not allowed. Param with key='best_n_estimators' was already logged with value='200' for run ID='c1669534ecc2451b995cd8fbae32a12d'. Attempted logging new value 'None'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

Which will create a new nested run for each individual
model and prevent parameter key collisions within the
tracking store.